# Conflict Resolution

## Conflicts in Git and why they are good

Git is very good at resolving modifications when merging branches and in most cases a git merge runs smooth and automatic. Then a merge commit appears without you even noticing.

But sometimes the same portion of the code/text is modified on two branches in two different ways and Git issues a conflict. Then you need to tell Git which version to keep (resolve it).

There are several ways to do that as we will see.

Please remember:

- It is good that Git conflicts exist: Git will not silently overwrite one of
  two differing modifications.
- Conflicts may look scary, but are not that bad after a little bit of
  practice. Also they are luckily rare.
- Don't be afraid of Git because of conflicts. You may not meet some conflicts
  using other systems because you simply can't do the kinds of things you do
  in Git.
- You can take human measures to reduce them.

## Preparing a conflict

We will make two branches, make two conflicting changes (both increase and
decrease the amount of cilantro), and later we will try to merge them
together.

Create two branches from `main`: one called `like-cilantro`, one called `dislike-cilantro`:

In [ ]:
%cd recipe

In [ ]:
%%bash
git branch like-cilantro main
git branch dislike-cilantro main

- On the two branches make **different modifications** to the amount of the **same ingredient**:

In [ ]:
%%bash
git checkout like-cilantro

In [ ]:
%%writefile ingredients.txt

* 2 tbsp cilantro
* 2 avocados
* 1 lime
* 1 tsp salt
* 1/2 onion

In [ ]:
%%bash
git add .
git commit -m "cilantro is so good!!"

In [ ]:
%%bash
git checkout dislike-cilantro

In [ ]:
%%writefile ingredients.txt

* 0.5 tbsp cilantro
* 2 avocados
* 1 lime
* 1 tsp salt
* 1/2 onion

In [ ]:
%%bash
git add .
git commit -m "cilantro is not good!!"

In [ ]:
%%bash
git checkout main

- On the branch `like-cilantro` we have the following change:

In [ ]:
%%bash
git diff main like-cilantro

- And on the branch `dislike-cilantro` we have the following change:

In [ ]:
%%bash
git diff main dislike-cilantro

## Merging conflicting changes

What do you expect will happen when we try to merge these two branches into main?

The first merge will work:

In [ ]:
%%bash
git checkout main
git status

In [ ]:
%%bash
git merge like-cilantro

But the second will fail:

In [ ]:
%%bash
git merge dislike-cilantro

Without conflict Git would have automatically created a merge commit,
but since there is a conflict, Git did not commit:

In [ ]:
%%bash
git status

Git won’t decide which to take and we need to decide. Observe how Git gives us clear instructions on how to move forward.

Let us inspect the conflicting file:

In [ ]:
%%bash
cat ingredients.txt

Git inserted resolution markers (the <<<<<<<, >>>>>>>, and =======).

Try also git diff:

In [ ]:
%%bash
git diff

`git diff` now only shows the conflicting part, nothing else.

## Aborting a conflicting merge

Sometimes you get a merge conflict but realize that you can't solve it without
talking to a colleague (who created the other change) first. What to do?

You can abort the merge and postponing conflict resolution by resetting the
repository to `HEAD` (last committed state):

```console
$ git merge --abort
```

The repository looks then exactly as it was before the merge.

## Resolving the conflict.

We will discuss 3 different ways to do this.

### Manual resolution

We have to edit the code/text between the resolution markers.  You
only have to care about what Git shows you: Git stages all files
without conflicts and leaves the files with conflicts unstaged.

> **Steps to resolve a conflict:**
> - Check status with `git status` and `git diff`.
> - Decide what you keep (the one, the other, or both or something else). Edit the file to do this.
    - Remove the resolution markers, if not already done.
    - The file(s) should now look exactly how you want them.
> - Check status with `git status` and `git diff`.
> - Tell Git that you have resolved the conflict with `git add ingredients.txt` (with certain plugin the editor may stage the change for you after you have removed the conflict markers).
> - Verify the result with `git status`.
> - Finally commit the merge with only `git commit`. Everything is pre-filled.


In [ ]:
%%bash
git diff

In [ ]:
%%writefile ingredients.txt

* 1.25 tbsp cilantro
* 2 avocados
* 1 lime
* 1 tsp salt
* 1/2 onion

In [ ]:
%%bash
git add ingredients.txt
git status

In [ ]:
%%bash
git commit -m "Merge branch 'dislike cilantro'"

### Resolve a conflict using mergetool

- Instead of resolving the conflict manually, use a visual tool
    (requires installing one of the [visual diff tools](https://coderefinery.github.io/installation/difftools/)):

> First check (look for `merge.tool`) and set up config diff tool:
> ```
> git config --list
> ```  
>     
> if no `merge.tool` configure, first list available tools:
> ```
> git mergetool --tool-help
> ```
>     
> select and set up the `merge.tool`:
> ```
> git config --global merge.tool vimdiff
> ```

```
$ git mergetool
```

<img src="img/mergetool.png" alt="" width="800" height="300"/>

 - Your current branch is left, the branch you merge is right, base is in the middle, and below is the diff.
 - After you are done, close and commit, `git add` is not needed when using `git mergetool`.

  If you have not instructed Git to avoid creating backups when using mergetool, then to be on
  the safe side there will be additional temporary files created. To remove those  you can do
  a git clean after the merging.

 To view what will be removed:

  ```
  $ git clean -n
  ```

  To remove:

  ```
  $ git clean -f
  ```

> **Note:**  
> To configure Git to avoid creating backups at all:
>  ```
> $ git config --global mergetool.keepBackup false
> ```

### Using "ours" or "theirs" strategy

- Sometimes you know that you want to keep "ours" version (version on this branch)
  or "theirs" (version on the merged branch).
- Then you do not have to resolve conflicts manually.
- See [merge strategies](https://git-scm.com/docs/merge-strategies).

Example:

```console
$ git merge -s recursive -Xours <branchname>  # merge and in doubt take the changes from current branch
```

Or:

```console
$ git merge -s recursive -Xtheirs <branchname>  # merge and in doubt take the changes from less-avocados branch
```


## Avoiding conflicts

- Human measures
  - Think and plan to which branch you will commit to.
  - Do not put unrelated changes on the same branch.
- Collaboration measures
  - Open an issue and discuss with collaborators before starting a long-living
    branch.
- Project layout measures
  - Modifying global data often causes conflicts.
  - Modular programming minimizes risk of conflicts.
- Technical measures
  - **Share your changes early and often** - this is one of the happy,
    rare circumstances when everyone doing the selfish thing (e.g. `git push` as
    early as practical) results in best case for everyone!
  - Pull/rebase often to keep up to date with upstream.
  - Resolve conflicts early.

# Summary
```
$ git merge --abort          # abort the merge and resetting the repository to HEAD (last committed state)
$ git mergetool --tool-help  # list available merge tools
$ git mergetool              # run merge conflict resolution tools to resolve merge conflicts
$ git clean -f               # remove additional temporary files
```

---

[back <](git-branches.ipynb)  [menu](../git-intro.ipynb)  [> next](git-online.ipynb) 

---